# Leaf Disease Detection

Learning goals:
- Exploratory data analysis is important
- Learn how to use [albumentations](https://github.com/albumentations-team/albumentations) augmentation library.
- Learn how to combine albumentations with TensorFlow Dataset API
- Learn how to write custom Grid Mask Augmentation class
- Learn about Generalized Average Pooling
- Learn about Label Smoothing
- Learn how to use Test Time Augmentation (TTA)

## Import Libraries

In [ ]:
# paths
import os
from os.path import join

# RNG and math
import random
import math
from math import ceil

# progress bar
from tqdm.notebook import tqdm # progress bar

# data processing
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

# deep learning
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, Lambda
from tensorflow.keras.applications import ResNet50, InceptionResNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

!pip install -q efficientnet
import efficientnet.tfkeras as efn

#plotting
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

# image manipulation
import cv2

# Augmentations
import albumentations
from albumentations import *
from albumentations.core.transforms_interface import DualTransform
from albumentations.augmentations import functional as F

# datasets
from kaggle_datasets import KaggleDatasets


## Main Modeling Parameters

In [ ]:
# Define size of the image to train on
IMAGE_X = 512
IMAGE_Y = 512
input_shape = (IMAGE_X, IMAGE_Y, 3)

EPOCHS = 50
LR = 0.00016
BATCH_SIZE = 16

LABEL_SMOOTHING_ALPHA=0.02
USE_GMP = True # Generalized Mean Pooling

SEED = 42

## Loading Data

In [ ]:
IMAGE_PATH = "../input/plant-pathology-2020-fgvc7/images/"
TEST_PATH = "../input/plant-pathology-2020-fgvc7/test.csv"
TRAIN_PATH = "../input/plant-pathology-2020-fgvc7/train.csv"
SUB_PATH = "../input/plant-pathology-2020-fgvc7/sample_submission.csv"
MODEL_PATH = "models/plant_pathology_model.h5"

sample_submission = pd.read_csv(SUB_PATH)
df_test = pd.read_csv(TEST_PATH)
df_train = pd.read_csv(TRAIN_PATH)
labels = list(df_train.columns[1:])
AUTO = tf.data.experimental.AUTOTUNE


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(SEED)

# Exploring the Data

### CSV files

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
print('Training set size:', len(df_train))
for label in labels:
    print(f"\t{label}: {df_train[df_train[label]==1].shape[0]}")
print('Test set size:', len(df_test))

### Images

In [ ]:
def load_image(image_id):
    file_path = "{}{}".format(image_id, ".jpg")
    image = cv2.imread(join(IMAGE_PATH, file_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

# plot some images
def plot_sample_images(preprocess_fn=None, nrows=4, ncols=4):
    fig, axs = plt.subplots(nrows, ncols,
                            figsize=(12,10)
                           )
    axs = axs.ravel() # make 1D array for easy plotting in for loop

    for i, image_id in enumerate(np.random.randint(len(df_train), size=nrows*ncols)):
        # show an image
        img = load_image(df_train['image_id'][image_id])
        
        if preprocess_fn:
            img = preprocess_fn(img)
        axs[i].imshow(img)
        axs[i].axis(False)
        label = df_train.loc[:, 'healthy':].iloc[image_id, :].idxmax()
        axs[i].set_title('{} | {}'.format(df_train['image_id'][image_id], label))
        plt.tight_layout()
plot_sample_images(nrows=4, ncols=4)

In the plot below you can check the exact values of each pixel. Notice that most of the pixels have high green and low blue values. However the spot on the leaf has high blue value.

In [ ]:
image = load_image(df_train['image_id'][0])
fig = px.imshow(image)
fig.show()

Lets have a closer look at the distribution of the channel values.

In [ ]:
# Code thanks to https://www.kaggle.com/tarunpaparaju/plant-pathology-2020-eda-models

n_samples = 100
train_images = df_train["image_id"][:n_samples].apply(load_image)

red_values = [np.mean(train_images[idx][:, :, 0]) for idx in range(len(train_images))]
green_values = [np.mean(train_images[idx][:, :, 1]) for idx in range(len(train_images))]
blue_values = [np.mean(train_images[idx][:, :, 2]) for idx in range(len(train_images))]
values = [np.mean(train_images[idx]) for idx in range(len(train_images))]

fig = ff.create_distplot([red_values, green_values, blue_values],
                         group_labels=["R", "G", "B"],
                         colors=["red", "green", "blue"])
fig.update_layout(title_text="Distribution of channel values")
fig

From the plot above we can see that the most pronounced color is green, which has the highest values. All other channels are shifted to the left with blue being the least pronounced.

### Problems with the data
Source: https://www.kaggle.com/c/plant-pathology-2020-fgvc7/discussion/154056

Some of the images have incosistent labels.
Images `Train_379` and `Train_1173` are the same images but have different labels.

In [ ]:
def plot_by_image_id(image_ids):

    fig, axs = plt.subplots(1, len(image_ids),
                                figsize=(12,10)
                               )
    axs = axs.ravel() # make 1D array for easy plotting in for loop

    for i, image_id in enumerate(image_ids):
        # show an image
        img = load_image(image_id)
        axs[i].imshow(img)
        axs[i].axis(False)
        img_ctgs = df_train[df_train['image_id'] == image_id][labels].values[0]
        img_label = np.argmax(df_train[df_train['image_id'] == image_id].loc[:, 'healthy':'scab'].values[0])
        axs[i].set_title('{} | {} | {}'.format(image_id, img_ctgs, labels[img_label]))
        plt.tight_layout()
        
plot_by_image_id(['Train_379', 'Train_1173'])

Images `Train_1` and `Train_171` are generated by the same image, but have different labels.

In [ ]:
plot_by_image_id(['Train_1', 'Train_171'])

### How to solve the inconsistencies in the labeling?
We have a couple of options for solving these inconsistencies:
1. Remove one duplicate images. However, now we are faced with questions:
    - Which image to remove?
    - Which label is the correct label?
2. Label Smoothing (used in this notebook). Label smoothings relaxes the confidence we have in the provided labels. We can control the relaxation with the parameter $\alpha$ and apply the following formula to our labels: $y_k^{LS}=y_K(1-\alpha) + \frac{\alpha}{K}$, where $y_k$ is $k$'th element of the one-hot encoded 'true' vector $y$ and $K$ is the number of classes.
    - Labels Smoothing makes the difference between prediction and 'true' label to be dependant on a constant $\alpha$
    - 'It encourages the activations of the penultimate layer to be close to the correct class template and equally distant to the templates of the incorrect classes.' [[source](https://medium.com/@nainaakash012/when-does-label-smoothing-help-89654ec75326)]
    - When does label smoothing help? [[paper](https://arxiv.org/pdf/1906.02629.pdf)], [[article](https://medium.com/@nainaakash012/when-does-label-smoothing-help-89654ec75326)]
3. Knowledge distillation (discussed in the [thread](https://www.kaggle.com/c/plant-pathology-2020-fgvc7/discussion/154056)). In knowledge distillation, you train a small student network with labels generated by a large teacher network. Intuitively you can understand it as relying less on the probably mistaken data and learning the labels from the images themselves.
    - Knowledge distillation in [Keras](https://arxiv.org/abs/2006.05525)
    - Original paper: [Distilling the Knowledge in a Neural Network](https://arxiv.org/abs/1503.02531)
    - [Knowledge Distillation: A Survey](https://arxiv.org/abs/2006.05525)

## Data Generators and Augmentation
For loading the data, we are going to use the TensorFlow dataset API. For the data augmnetation, we are going to use Albumentations [[Docs](https://albumentations.ai/docs/getting_started/installation/)] and show how to use it with TensorFlow Dataset API.

In [ ]:
class GridMask(DualTransform):
    
    """GridMask augmentation for image classification and object detection.
    
    Author: Qishen Ha
    Email: haqishen@gmail.com
    2020/01/29

    Args:
        num_grid (int): number of grid in a row or column.
        fill_value (int, float, lisf of int, list of float): value for dropped pixels.
        rotate ((int, int) or int): range from which a random angle is picked. If rotate is a single int
            an angle is picked from (-rotate, rotate). Default: (-90, 90)
        mode (int):
            0 - cropout a quarter of the square of each grid (left top)
            1 - reserve a quarter of the square of each grid (left top)
            2 - cropout 2 quarter of the square of each grid (left top & right bottom)

    Targets:
        image, mask

    Image types:
        uint8, float32

    Reference:
    |  https://arxiv.org/abs/2001.04086
    |  https://github.com/akuxcw/GridMask
    """

    def __init__(self, num_grid=3, fill_value=0, rotate=0, mode=0, always_apply=False, p=0.5):
        super(GridMask, self).__init__(always_apply, p)
        if isinstance(num_grid, int):
            num_grid = (num_grid, num_grid)
        if isinstance(rotate, int):
            rotate = (-rotate, rotate)
        self.num_grid = num_grid
        self.fill_value = fill_value
        self.rotate = rotate
        self.mode = mode
        self.masks = None
        self.rand_h_max = []
        self.rand_w_max = []

    def init_masks(self, height, width):
        if self.masks is None:
            self.masks = []
            n_masks = self.num_grid[1] - self.num_grid[0] + 1
            for n, n_g in enumerate(range(self.num_grid[0], self.num_grid[1] + 1, 1)):
                grid_h = height / n_g
                grid_w = width / n_g
                this_mask = np.ones((int((n_g + 1) * grid_h), int((n_g + 1) * grid_w))).astype(np.uint8)
                for i in range(n_g + 1):
                    for j in range(n_g + 1):
                        this_mask[
                             int(i * grid_h) : int(i * grid_h + grid_h / 2),
                             int(j * grid_w) : int(j * grid_w + grid_w / 2)
                        ] = self.fill_value
                        if self.mode == 2:
                            this_mask[
                                 int(i * grid_h + grid_h / 2) : int(i * grid_h + grid_h),
                                 int(j * grid_w + grid_w / 2) : int(j * grid_w + grid_w)
                            ] = self.fill_value
                
                if self.mode == 1:
                    this_mask = 1 - this_mask

                self.masks.append(this_mask)
                self.rand_h_max.append(grid_h)
                self.rand_w_max.append(grid_w)

    def apply(self, image, mask, rand_h, rand_w, angle, **params):
        h, w = image.shape[:2]
        mask = F.rotate(mask, angle) if self.rotate[1] > 0 else mask
        mask = mask[:,:,np.newaxis] if image.ndim == 3 else mask
        image *= mask[rand_h:rand_h+h, rand_w:rand_w+w].astype(image.dtype)
        return image

    def get_params_dependent_on_targets(self, params):
        img = params['image']
        height, width = img.shape[:2]
        self.init_masks(height, width)

        mid = np.random.randint(len(self.masks))
        mask = self.masks[mid]
        rand_h = np.random.randint(self.rand_h_max[mid])
        rand_w = np.random.randint(self.rand_w_max[mid])
        angle = np.random.randint(self.rotate[0], self.rotate[1]) if self.rotate[1] > 0 else 0

        return {'mask': mask, 'rand_h': rand_h, 'rand_w': rand_w, 'angle': angle}

    @property
    def targets_as_params(self):
        return ['image']

    def get_transform_init_args_names(self):
        return ('num_grid', 'fill_value', 'rotate', 'mode')

Here we define a dictionary with different kinds of augmentations. The key is the name of the augmentation functions set, and the value is a single Albumentations class. You can use each Albumentations class one by one or use `Compose` to string several augmentations together. If you would like to choose only one augmentation at a time, you can use `OneOf`.

In [ ]:

transform = {
    'train': Compose([
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
#         RandomRotate90(p=0.5),
        GaussianBlur(blur_limit=(3, 7), always_apply=False, p=0.5),
        ShiftScaleRotate(shift_limit=0.0625,
                        scale_limit=0.1,
                        rotate_limit=45,
                        interpolation=cv2.INTER_LINEAR,
                        border_mode=cv2.BORDER_REFLECT_101,
                        always_apply=False,
                        p=0.5,),
        RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5, always_apply=False),
        GridMask(num_grid=4, rotate=15, p=0.5)
    ]),
    
    'vertical_flip': VerticalFlip(always_apply=True),
    
    'horizontal_flip': HorizontalFlip(always_apply=True),
    
    'vh_flip': Compose([
        VerticalFlip(always_apply=True),
        HorizontalFlip(always_apply=True)
    ]),
    
    'brightness': RandomBrightnessContrast(
        brightness_limit=0.2,
        contrast_limit=0.2,
        p=0.5, always_apply=True),
    
    'blur': GaussianBlur(blur_limit=(3, 7), always_apply=True),
    
    'shift_scale_rotate' : ShiftScaleRotate(
                        shift_limit=0.0625,
                        scale_limit=0.1,
                        rotate_limit=45,
                        interpolation=cv2.INTER_LINEAR,
                        border_mode=cv2.BORDER_REFLECT_101,
                        always_apply=True)
}

def preprocess(df, test=False):
    paths = df.image_id.apply(lambda x: IMAGE_PATH + x + '.jpg').values
    labels = df.loc[:,'healthy':].values
    if test==False:
        return paths, labels
    else:
        return paths
    
def decode_image(filename, label=None, image_size=(IMAGE_X, IMAGE_Y)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3) 
    image = tf.image.resize(image, image_size)
    image = tf.cast(image, tf.float32)
#     image = tf.image.per_image_standardization(image)
    image = tf.divide(image, 255.)
    if label is None:
        return image
    else:
        return image, label
    
def data_augment(image, label=None, seed=SEED):
    image = tf.image.rot90(image,k=np.random.randint(4))
    image = tf.image.random_flip_left_right(image, seed=seed)
    image = tf.image.random_flip_up_down(image, seed=seed)
    if label is None:
        return image
    else:
        return image, label

def get_augmentation_func(set_name='train'):
    """
    Given augmentation class name returs a function to augment images
    and the same function wrapped into tf.pyfunction, so it
    can beused in the tensorflow computation graph
    """
    def albu(image):
        transforms = transform[set_name]
        image = transforms(image=image.numpy())['image']
        image = tf.cast(image, tf.float32)
        return image

    def albu_fn(image,label=None):
        [image,] = tf.py_function(albu, [image], [tf.float32])
        if label is None:
            return image
        else:
            return image, label
    return albu, albu_fn

In [ ]:
def plot_transform(num_images=7, set_name='train'):
    fig, axs = plt.subplots(nrows=2, ncols=num_images, figsize=(30,10))
    for i in range(0, num_images):
        image_id = np.random.randint(len(df_train))
        path, _ = preprocess(df_train.iloc[image_id: image_id+1])
        image = decode_image(filename=path[0])
        axs[0, i].imshow(image)
        albu, albu_fn = get_augmentation_func(set_name)
        image = albu(image)
        axs[1, i].imshow(image)

#### Training augmenation

In [ ]:
plot_transform(5, 'train')

In [ ]:
train, valid = train_test_split(df_train, test_size=0.2, random_state=SEED)

albu, albu_fn = get_augmentation_func('train')

train_dataset = (tf.data.Dataset
    .from_tensor_slices(preprocess(train))
    .map(decode_image, num_parallel_calls=AUTO)
#     .map(data_augment, num_parallel_calls=AUTO)
    .map(albu_fn, num_parallel_calls=AUTO)
    .shuffle(SEED)
    .batch(BATCH_SIZE, drop_remainder=True)
    .repeat()
    .prefetch(AUTO))

valid_dataset = (tf.data.Dataset
    .from_tensor_slices(preprocess(valid))
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO))

test_dataset = (tf.data.Dataset
    .from_tensor_slices(preprocess(df_test,test=True))
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE))

## Modeling

In [ ]:
from tensorflow.keras.applications import MobileNet, DenseNet121

# Generalized Mean Pooling https://paperswithcode.com/method/generalized-mean-pooling 
class GeneralizedMeanPool(tf.keras.layers.Layer):
    def __init__(self):
        super(GeneralizedMeanPool, self).__init__()
        self.gm_exp = tf.Variable(3.0, dtype = tf.float32)

    def call(self, inputs):
        pool = (tf.reduce_mean(tf.abs(inputs ** (self.gm_exp)), 
            axis = [1, 2], 
            keepdims = False) + 1.e-7) ** (1. / self.gm_exp)
        return pool


def create_model(input_shape, train_conv_layers=True, use_gmp=False):
    input_ = Input(shape = input_shape)

    #Create and complite model and show summary
    
    x_model = DenseNet121(weights='imagenet',
                       include_top=False,
                       input_tensor=input_,
                       pooling=None,
                       classes=None)
    
    for layer in x_model.layers:
        layer.trainable = train_conv_layers
    
    # GMP
    x = x_model.output
    if use_gmp:
        x = GeneralizedMeanPool()(x)
    
    #output 
    output = Dense(4, activation='softmax', name='plan_diseases')(x)
   
    
    #model 
    model = Model(inputs = x_model.input, outputs=output)
    
    return model

In [ ]:
model = create_model(input_shape, train_conv_layers=True, use_gmp=USE_GMP)

# label smoothing could be used by passing additional
# parameter to the keras loss
ls_loss = tf.keras.losses.CategoricalCrossentropy(
    label_smoothing=LABEL_SMOOTHING_ALPHA,
    name='categorical_crossentropy'
)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
              loss=ls_loss,
              metrics = ['categorical_accuracy'])

### Callbacks

In [ ]:
best_w = ModelCheckpoint('plant_best.h5',
                                monitor='val_loss',
                                verbose=0,
                                save_best_only=True,
                                save_weights_only=True,
                                mode='auto',
                                period=1)

last_w = ModelCheckpoint('plant_last.h5',
                               monitor='val_loss',
                                verbose=0,
                                save_best_only=False,
                              mode='auto',
                                period=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1,cooldown=1)
callbacks = [best_w, last_w, reduce_lr]

## Training

In [ ]:
STEPS_PER_EPOCH = train.shape[0] // BATCH_SIZE
history = model.fit(train_dataset,
          steps_per_epoch=STEPS_PER_EPOCH,
          epochs=EPOCHS, 
          verbose=1,
          callbacks=callbacks,
          validation_data=valid_dataset,
          use_multiprocessing=False)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def visualize_training_process(history):
    """ 
    Visualize loss and accuracy from training history
    
    :param history: A Keras History object
    """
    history_df = pd.DataFrame(history.history)
    epochs = np.arange(1, len(history_df) + 1)
    fig = make_subplots(2, 1)
    fig.append_trace(go.Scatter(x=epochs, y=history_df['categorical_accuracy'], mode='lines+markers', name='Accuracy Train'), row=1, col=1)
    fig.append_trace(go.Scatter(x=epochs, y=history_df['val_categorical_accuracy'], mode='lines+markers', name='Accuracy Val'), row=1, col=1)
    
    fig.append_trace(go.Scatter(x=epochs, y=history_df['loss'], mode='lines+markers', name='Loss Train'), row=2, col=1)
    fig.append_trace(go.Scatter(x=epochs, y=history_df['val_loss'], mode='lines+markers', name='Loss Val'), row=2, col=1)
    
    fig.update_layout( xaxis_title="Epochs", template="plotly_white")
    
    return fig
visualize_training_process(history)

In [ ]:
# load the best model
model.load_weights('plant_best.h5')

In [ ]:
# Make predictions without Test Time Augmentation
test_pred = model.predict(test_dataset, verbose=1)
submission_df = pd.read_csv(SUB_PATH)
submission_df[['healthy', 'multiple_diseases', 'rust', 'scab']] = test_pred
submission_df.to_csv('submission.csv', index=False)
pd.Series(np.argmax(submission_df[labels].values,axis=1)).value_counts()

In [ ]:
# TTA all
test_pred_tta = np.zeros((len(df_test), 4))
# Choose what augmentations to use during predictions
augmentation_sets = [
    'vertical_flip',
    'horizontal_flip',
    'vh_flip',
    'brightness',
    'blur',
    'shift_scale_rotate'
]
for i, set_name in enumerate(augmentation_sets):
    _, albu_fn = get_augmentation_func(set_name)
    test_dataset_tta = (
        tf.data.Dataset.from_tensor_slices(preprocess(df_test, test=True))
                       .map(decode_image, num_parallel_calls=AUTO)
                       .map(albu_fn, num_parallel_calls=AUTO)    
                       .batch(BATCH_SIZE))
    
    test_pred_tta += model.predict(test_dataset_tta, verbose=1)
    
submission_df = pd.read_csv(SUB_PATH)
submission_df[['healthy', 'multiple_diseases', 'rust', 'scab']] = test_pred_tta / len(augmentation_sets)
submission_df.to_csv('submission_tta_all.csv', index=False)
pd.Series(np.argmax(submission_df[labels].values,axis=1)).value_counts()

In [ ]:
# TTA simple
test_pred_tta = np.zeros((len(df_test), 4))
# Choose what augmentations to use during predictions
augmentation_sets = [
    'vertical_flip',
    'horizontal_flip',
    'vh_flip',
#     'brightness',
#     'blur',
#     'shift_scale_rotate'
]
for i, set_name in enumerate(augmentation_sets):
    _, albu_fn = get_augmentation_func(set_name)
    test_dataset_tta = (
        tf.data.Dataset.from_tensor_slices(preprocess(df_test, test=True))
                       .map(decode_image, num_parallel_calls=AUTO)
                       .map(albu_fn, num_parallel_calls=AUTO)    
                       .batch(BATCH_SIZE))
    
    test_pred_tta += model.predict(test_dataset_tta, verbose=1)
    
submission_df = pd.read_csv(SUB_PATH)
submission_df[['healthy', 'multiple_diseases', 'rust', 'scab']] = test_pred_tta / len(augmentation_sets)
submission_df.to_csv('submission_tta_simple.csv', index=False)
pd.Series(np.argmax(submission_df[labels].values,axis=1)).value_counts()

In [ ]:
# # Evaluate performance of model by plotting confusion matrix
# from sklearn.metrics import confusion_matrix

# # see http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
# import itertools

# def accuracy(y, y_pred):
#     return np.sum(y == y_pred)/len(y)

# def plot_confusion_matrix(cm, labels=None, title='Confusion Matrix'):
#     import plotly.figure_factory as ff

#     x = labels
#     y = x

#     # change each element of z to type string for annotations
#     z_text = [[str(y) for y in x] for x in cm]

#     # set up figure 
#     fig = ff.create_annotated_heatmap(cm, x=x, y=y, annotation_text=z_text, colorscale='YlGnBu', showscale=True)

#     # add title
#     fig.update_layout(title_text=title,
#                       #xaxis = dict(title='x'),
#                       #yaxis = dict(title='x')
#                      )

#     # add custom xaxis title
#     fig.add_annotation(dict(font=dict(color="black",size=14),
#                             x=0.5,
#                             y=-0.15,
#                             showarrow=False,
#                             text="Predicted value",
#                             xref="paper",
#                             yref="paper"))

#     # add custom yaxis title
#     fig.add_annotation(dict(font=dict(color="black",size=14),
#                             x=-0.35,
#                             y=0.5,
#                             showarrow=False,
#                             text="Real value",
#                             textangle=-90,
#                             xref="paper",
#                             yref="paper"))

#     # adjust margins to make room for yaxis title
#     fig.update_layout(margin=dict(t=100, l=200), width=700, height=600)
#     fig.show()
    
# # predict labels from validation set
# y_pred = model.predict(val_gen)
# # convert data to label number

 
# y_true = np.argmax(Y_val.values, axis=1) 

# # compute the confusion matrix
# cm = confusion_matrix(y_true, y_pred) 

# plot_confusion_matrix(cm, labels, title='Confusion_matrix Validation Set (acc={:.3f})'.format(accuracy(y_true, y_pred)))